In [1]:
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [5]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_rides_data

from_year = 2024
# to_year = datetime.now().year
to_year = 2024
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_rides_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2024 to 2024
File already exists for 2024-01.
Loading data for 2024-01...
Total records: 1,888,085
Valid records: 1,880,623
Records dropped: 7,462 (0.40%)
Successfully processed data for 2024-01.
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 2,121,501
Valid records: 2,115,129
Records dropped: 6,372 (0.30%)
Successfully processed data for 2024-02.
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 2,663,295
Valid records: 2,654,825
Records dropped: 8,470 (0.32%)
Successfully processed data for 2024-03.
File already exists for 2024-04.
Loading data for 2024-04...
Total records: 3,217,063
Valid records: 3,207,286
Records dropped: 9,777 (0.30%)
Successfully processed data for 2024-04.
File already exists for 2024-05.
Loading data for 2024-05...
Total records: 4,230,360
Valid records: 4,215,930
Records dropped: 14,430 (0.34%)
Successfully processed data for 2024-05.
File already exists for 2024-06.
Loading data fo

In [6]:
rides

,pickup_datetime,pickup_location_id
0,2024-01-22 18:43:19.012,7954.12
1,2024-01-11 19:19:18.721,6771.13
2,2024-01-30 19:17:41.693,5659.11
3,2024-01-27 11:27:01.759,6771.13
4,2024-01-16 15:15:41.000,7443.01
...,...,...
44233324,2024-12-21 19:08:07.762,4386.07
44233325,2024-12-17 18:59:56.045,6925.09
44233326,2024-12-18 06:44:59.074,6717.06
44233327,2024-12-20 20:53:58.339,4019.06


In [7]:
rides.shape

(44233329, 2)

In [8]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [9]:
ts_data.shape

(20879568, 3)

In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20879568 entries, 0 to 20879567
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  object        
 2   rides               int16         
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 358.4+ MB


In [11]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2024-01-01 00:00:00,190 Morgan,0
1,2024-01-01 01:00:00,190 Morgan,0
2,2024-01-01 02:00:00,190 Morgan,0
3,2024-01-01 03:00:00,190 Morgan,0
4,2024-01-01 04:00:00,190 Morgan,0
...,...,...,...
20879563,2024-12-31 19:00:00,Shop Morgan,0
20879564,2024-12-31 20:00:00,Shop Morgan,0
20879565,2024-12-31 21:00:00,Shop Morgan,0
20879566,2024-12-31 22:00:00,Shop Morgan,0


In [12]:
import hopsworks

api_key = os.getenv('HOPSWORK_API_KEY')  
project_name = os.getenv('HOPSWORK_PROJECT_NAME')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-10 12:56:25,257 INFO: Initializing external client
2025-05-10 12:56:25,257 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 12:56:31,364 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Successfully connected to Hopsworks project: MLOPS_End_To_End_Project


In [17]:
feature_store = project.get_feature_store()

In [18]:
FEATURE_GROUP_NAME = "citibike_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [19]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description = "Time-series data at hourly frequency",
    primary_key = ["pickup_location_id","pickup_hour"],
    event_time = "pickup_hour"
)

In [20]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214680/fs/1203298/fg/1454606


Uploading Dataframe: 100.00% |██████████| Rows 20879568/20879568 | Elapsed Time: 1:00:00 | Remaining Time: 00:00


Launching job: citibike_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214680/jobs/named/citibike_hourly_feature_group_1_offline_fg_materialization/executions


(Job('citibike_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)